---
### 네이버 지도 API 이용 - 데이터 크롤링

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
pd.set_option('display.max_columns',50) # DataFrame truncation 없이 보기

import re
import datetime
import time
import requests
import math

In [2]:
# 네이버 API 인증 ID
client_id = 'sDYPCi0TlT6FFoKVbOTz'

# 네이버 API 인증 PW
client_secret = 'lyX1n9_ht3'

In [3]:
# 네이버 API로 지도검색 정보를 요청하는 함수
# request_naver_api(검색어, 시작번호, 조회개수)

def request_naver_api(keyword, start, display):
    
    # 기본 URL
    url = 'https://openapi.naver.com/v1/search/local.json'
    
    # 헤더에 담을 인증키 정보
    header = {'X-Naver-Client-Id' : client_id, 'X-Naver-Client-Secret' : client_secret }

    # 쿼리스트링
    queryString = {
          'query' : '서울 ' + keyword
        , 'start' : str(start)
        , 'display' : str(display)
    }
    
    try: 
        # request 보내서 response_naver에 저장
        response_naver = requests.get(url, headers=header, params = queryString)
        if response_naver.status_code == 200: # 정상처리되면
            print ("[%s] URL Request Success" % datetime.datetime.now()) # 정상출력
            return response_naver.json() # 실행결과 dict 객체로 변환하여 반환
    except Exception as e: # 에러 발생시
        print(e) # 에러 메세지 출력
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url)) # 에러 URL 출력
        return None

In [4]:
# '서울 준오헤어'로 검색, 1번 데이터부터, 30개 요청
dict_result = request_naver_api('준오헤어', 1, 1)

[2020-04-06 19:58:33.119191] URL Request Success


In [5]:
# 총 검색결과 수 뽑기
total_count = dict_result['total'] 
total_count

106

In [6]:
# 최대 요청 개수인 30으로 나누어서 반올림
# 요청할 회수
request_count = math.ceil(total_count / 30)
request_count

4

In [7]:
# 요청 회수만큼 URL 만들어서 리스트에 저장
dict_list_naver=[]
for i in range(request_count):
    dict_list_naver.append(request_naver_api('준오헤어', 1 + 30 * i, 30))

dict_list_naver

[2020-04-06 19:58:33.284705] URL Request Success
[2020-04-06 19:58:33.427319] URL Request Success
[2020-04-06 19:58:33.561990] URL Request Success
[2020-04-06 19:58:33.687661] URL Request Success


[{'lastBuildDate': 'Mon, 06 Apr 2020 19:58:32 +0900',
  'total': 106,
  'start': 1,
  'display': 30,
  'items': [{'title': '<b>준오헤어</b> 강남2호점',
    'link': 'https://junohairgangnam.modoo.at/',
    'category': '생활,편의>미용실',
    'description': '',
    'telephone': '02-536-6605',
    'address': '서울특별시 서초구 서초동 1305',
    'roadAddress': '서울특별시 서초구 강남대로 441 서산빌딩 3층',
    'mapx': '314018',
    'mapy': '544856'},
   {'title': '<b>준오헤어</b> 목동41타워점',
    'link': 'https://junohairmokdong41.modoo.at/',
    'category': '생활,편의>미용실',
    'description': '',
    'telephone': '02-2649-0605',
    'address': '서울특별시 양천구 목동 917-9',
    'roadAddress': '서울특별시 양천구 목동동로 293 현대41타워 3층',
    'mapx': '300831',
    'mapy': '547930'},
   {'title': '<b>준오헤어</b> 노원사거리점',
    'link': 'http://blog.naver.com/hari4919',
    'category': '생활,편의>미용실',
    'description': '',
    'telephone': '02-938-0605',
    'address': '서울특별시 노원구 상계동 712',
    'roadAddress': '서울특별시 노원구 동일로 1413 남성빌딩 3층 준오헤어',
    'mapx': '317258',
    'mapy'

In [8]:
# 검색결과인 'items' key에 해당하는 value 만 뽑기
item_list_naver = list(map(lambda x: x['items'], dict_list_naver))

In [9]:
# 리스트에 저장된 각각의 데이터를 DataFrame으로 말기
df_list_naver = list(map(lambda x: DataFrame(x), item_list_naver))

In [10]:
# 리스트의 모든 DF를 하나로 합치기
df_all_naver = pd.concat(df_list_naver, ignore_index=True)

In [11]:
# 필요한 컬럼으로 재색인
df_all_naver = df_all_naver.reindex(columns = ['title', 'telephone', 'address', 'roadAddress'])

In [12]:
# 'title'에 들어간 '<br>', '</br>' 등 html 태그 지우기
df_all_naver['title'] = np.array(list(map(lambda x: re.sub('</\w+>|<\w+>','',x), df_all_naver['title'].values)))

In [13]:
# 결과 확인
# 105개 total_count만큼 잘 조회가 되었다.
# 그런데 준오헤어가 아닌 데이터들이 포함된 것 같다.

df_all_naver.tail(2)

,title,telephone,address,roadAddress
104,준오뷰티,02-548-0605,서울특별시 강남구 청담동 63-14 준오헤어청담동사옥 5층,서울특별시 강남구 삼성로 728 애브뉴준오 5층
105,비에비스나무병원,1577-7502,서울특별시 강남구 논현동 175-10,서울특별시 강남구 논현로 627 준오헤어빌딩


In [14]:
# 80번부터 끝까지 조회
# 97번부터 이상한 데이터가 들어가있다.

df_all_naver.iloc[80:]

,title,telephone,address,roadAddress
80,준오헤어 양재2호점,02-2057-0605,서울특별시 서초구 양재동 12-3,서울특별시 서초구 남부순환로350길 4 2층
81,준오헤어 현대시티몰가든파이브점,02-2157-0605,서울특별시 송파구 문정동 634,서울특별시 송파구 충민로 66 현대시티몰 가든파이브점 아울렛관 5층
82,준오헤어 신내홈플러스점,02-3422-0605,서울특별시 중랑구 신내동 645,서울특별시 중랑구 신내로 201
83,준오헤어 반포점,02-3478-0605,서울특별시 서초구 반포동 56-2,서울특별시 서초구 서초중앙로 239 2층 준오헤어 반포점
84,준오헤어 목동 이마트점,02-2062-0605,서울특별시 양천구 목동 962,서울특별시 양천구 오목로 299 이마트 지하2층
85,준오헤어 강남1호점,02-556-7488,서울특별시 강남구 역삼동 816-4,서울특별시 강남구 강남대로96길 5 3층
86,준오헤어 건대역1호점,02-426-0605,서울특별시 광진구 화양동 4-20,서울특별시 광진구 능동로 110
87,준오헤어 은평뉴타운점,02-383-0605,서울특별시 은평구 진관동 64,서울특별시 은평구 진관3로 11
88,준오헤어 강남화이트점,02-592-0605,서울특별시 서초구 서초동 1317-7,서울특별시 서초구 서초대로77길 13 3층
89,준오헤어 가든도산공원점,02-544-0605,서울특별시 강남구 논현동 92,서울특별시 강남구 도산대로 318 SB타워 별관 A동 3층 준오헤어
